# Inference

### For the 3 models, try to predict on some messages


In [2]:
!pip3 install --upgrade openai -q

In [28]:
import json, openai, time, pandas, random, getpass
from openai import cli
from types import SimpleNamespace
from sklearn.model_selection import train_test_split
from file_read_backwards import FileReadBackwards

work_dir = "/Users/eric.pinzur/Documents/slackbot2000"
openai.api_key = getpass.getpass(prompt="Please enter your OpenAI API Key")

### Model IDs

In [3]:
next_message_model = 'ada:ft-personal:next-message-user-full-kaskada-2023-08-03-15-42-55'

next_reaction_model = 'ada:ft-personal:next-reaction-full-kaskada-2023-08-03-17-03-13'

next_users_model = 'ada:ft-personal:next-users-in-window-full-kaskada-2023-08-03-18-17-36'

## Next Message User

For a set of recent messages, try to predict the next user that will reply.

This is a classification problem, so we can get log probabilities

In [2]:
nm_completion_separator = ""

nm_train = pandas.read_json(f'{work_dir}/next_message_user_cleaned_train.jsonl', lines=True)
nm_valid = pandas.read_json(f'{work_dir}/next_message_user_cleaned_valid.jsonl', lines=True)

### With Training Data

In [4]:
for i in range(10):
    prompt = nm_train['prompt'][i]
    completion = nm_train['completion'][i]

    res = openai.Completion.create(model=next_message_model, prompt=prompt, max_tokens=1, temperature=0, logprobs=3)
    choice = res['choices'][0]['text']
    log_probs = res['choices'][0]['logprobs']['top_logprobs'][0]

    print(f'Choice: {choice}, Actual: {completion}, Log Probabilites:')
    print(log_probs) 
    print()

Choice:  1, Actual: 1, Log Probabilites:
{
  " 1": -0.9657385,
  " 2": -1.4050677,
  " 4": -1.596037
}

Choice:  1, Actual: 2, Log Probabilites:
{
  " 1": -1.0685356,
  " 2": -1.5028064,
  " 4": -1.3944669
}

Choice:  1, Actual: 2, Log Probabilites:
{
  " 1": -1.050273,
  " 2": -1.5859404,
  " 4": -1.3914024
}

Choice:  4, Actual: 1, Log Probabilites:
{
  " 1": -1.2921542,
  " 2": -1.9428157,
  " 4": -1.2020205
}

Choice:  4, Actual: 1, Log Probabilites:
{
  " 1": -1.3886855,
  " 2": -1.8444825,
  " 4": -1.2994654
}

Choice:  1, Actual: 3, Log Probabilites:
{
  " 1": -0.8068829,
  " 4": -1.8223191,
  " 6": -1.9705716
}

Choice:  1, Actual: 1, Log Probabilites:
{
  " 1": -0.81668377,
  " 2": -1.960412,
  " 4": -1.8723706
}

Choice:  1, Actual: 4, Log Probabilites:
{
  " 1": -0.9468565,
  " 4": -1.9926653,
  " 6": -1.6005487
}

Choice:  1, Actual: 4, Log Probabilites:
{
  " 1": -1.0770806,
  " 4": -1.8575277,
  " 6": -1.674056
}

Choice:  1, Actual: 1, Log Probabilites:
{
  " 1": -1.0844

### With Validation Data

In [15]:
for i in range(10):
    prompt = nm_valid['prompt'][i]
    completion = nm_valid['completion'][i]

    res = openai.Completion.create(model=next_message_model, prompt=prompt, max_tokens=1, temperature=0, logprobs=3)
    choice = res['choices'][0]['text']
    log_probs = res['choices'][0]['logprobs']['top_logprobs'][0]

    print(f'Choice: {choice}, Actual: {completion}, Log Probabilites:')
    print(log_probs) 
    print()

With Validation data:

Choice:  1, Actual: 2, Log Probabilites:
{
  " 1": -1.1703091,
  " 2": -1.7772275,
  " 4": -1.2289906
}

Choice:  1, Actual: 4, Log Probabilites:
{
  " 1": -0.9397264,
  " 4": -1.7467943,
  " 6": -1.9116483
}

Choice:  4, Actual: 1, Log Probabilites:
{
  " 1": -1.6808176,
  " 2": -1.3595152,
  " 4": -1.3301457
}

Choice:  4, Actual: 1, Log Probabilites:
{
  " 1": -1.2772946,
  " 4": -1.2063621,
  " 6": -1.9627421
}

Choice:  4, Actual: 1, Log Probabilites:
{
  " 1": -1.5786409,
  " 4": -1.0844089,
  " 6": -1.70502
}

Choice:  1, Actual: 2, Log Probabilites:
{
  " 1": -1.3150066,
  " 4": -1.3966527,
  " 6": -1.5908817
}

Choice:  1, Actual: 1, Log Probabilites:
{
  " 1": -0.8575319,
  " 2": -2.4999099,
  " 4": -1.3080171
}

Choice:  1, Actual: 6, Log Probabilites:
{
  " 1": -0.82480097,
  " 4": -1.3392587,
  " 6": -2.303694
}

Choice:  1, Actual: 5, Log Probabilites:
{
  " 1": -0.7997786,
  " 4": -1.4031092,
  " 6": -2.1099305
}

Choice:  1, Actual: 3, Log Probabi

## Next Reaction

For a set of recent messages, try to predict the reaction (if any) to the next message.

In [4]:
nr_completion_separator = " END"

nr_train = pandas.read_json(f'{work_dir}/next_reaction_train.jsonl', lines=True)
nr_valid = pandas.read_json(f'{work_dir}/next_reaction_valid.jsonl', lines=True)

### With Training Data

In [22]:
examples = 1

random.seed()
pos_count = 0
neg_count = 0

i = 0
while True:
    if pos_count == examples and neg_count == examples:
        break

    i += random.randint(0, 10)
    prompt = nr_train['prompt'][i]
    completion = nr_train['completion'][i]

    pos = completion != f' {nr_completion_separator}'
    if pos:
        if pos_count < examples:
            pos_count += 1
        else:
            continue
    else:
        if neg_count < examples:
            neg_count += 1
        else: 
            continue

    res = openai.Completion.create(model=next_reaction_model, prompt=prompt, max_tokens=20, top_p=0.5, stop=nr_completion_separator, logprobs=1, n=2)
    print(res)
    choice = res['choices'][0]['text']
    log_probs = res['choices'][0]['logprobs']['top_logprobs'][0]

    print(f'Choice: {choice}, Actual: {completion}, Log Probabilites:')
    print(log_probs) 
    print()

{
  "id": "cmpl-7jpAUdMimUX0BgS5GvO3c9lecHtGZ",
  "object": "text_completion",
  "created": 1691155806,
  "model": "ada:ft-personal:next-reaction-full-kaskada-2023-08-03-17-03-13",
  "choices": [
    {
      "text": " ",
      "index": 0,
      "logprobs": {
        "tokens": [
          " "
        ],
        "token_logprobs": [
          -0.046379454
        ],
        "top_logprobs": [
          {
            " ": -0.046379454
          }
        ],
        "text_offset": [
          688
        ]
      },
      "finish_reason": "stop"
    },
    {
      "text": " ",
      "index": 1,
      "logprobs": {
        "tokens": [
          " "
        ],
        "token_logprobs": [
          -0.046379454
        ],
        "top_logprobs": [
          {
            " ": -0.046379454
          }
        ],
        "text_offset": [
          688
        ]
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 170,
    "completion_tokens": 2,
    "total_tokens": 1

### With Validation Data

In [30]:
examples = 5

random.seed()
pos_count = 0
neg_count = 0

i = 0
while True:
    if pos_count == examples and neg_count == examples:
        break

    i += random.randint(0, 10)
    prompt = nr_valid['prompt'][i]
    completion = nr_valid['completion'][i]

    pos = completion != f' {nr_completion_separator}'
    if pos:
        if pos_count < examples:
            pos_count += 1
        else:
            continue
    else:
        if neg_count < examples:
            neg_count += 1
        else: 
            continue

    res = openai.Completion.create(model=next_reaction_model, prompt=prompt, max_tokens=20, temperature=0, stop=nr_completion_separator)
    choice = res['choices'][0]['text']

    print(f'Choice: {choice}, Actual: {completion}')
    print()

Choice:  , Actual:   END

Choice:  , Actual:   END

Choice:  , Actual:   END

Choice:  , Actual:   END

Choice:  , Actual:   END

Choice:  , Actual:  [{"name": "+1", "users": ["UU5C7MNMA"], "count": 1}] END

Choice:  , Actual:  [{"name": "+1", "users": ["U03RNK543HC"], "count": 1}] END

Choice:  , Actual:  [{"name": "+1", "users": ["U012CLUV1KJ"], "count": 1}] END

Choice:  , Actual:  [{"name": "sweat_smile", "users": ["U012CLUV1KJ"], "count": 1}] END

Choice:  , Actual:  [{"name": "+1", "users": ["U012CLUV1KJ"], "count": 1}] END



### Compute Metrics

In [7]:
#from IPython.display import clear_output

#nr_valid_with_predictions = pandas.read_json(f'{work_dir}/next_reaction_valid.jsonl', lines=True)

#total = len(nr_valid_with_predictions)
#for i in range(total):
#    clear_output(wait=True)
#    print(f'Processing row {i} of {total}...')
#    res = openai.Completion.create(model=next_users_model, prompt=nr_valid_with_predictions['prompt'][i], max_tokens=50, stop=nr_completion_separator)
#    nr_valid_with_predictions.at[i, "prediction"] = res['choices'][0]['text']

#nr_valid_with_predictions.to_json(f'{work_dir}/next_reaction_valid_with_pred.jsonl', lines=True, orient='records')
#nr_valid_with_predictions

Processing row 3934 of 3935...


,prompt,completion,prediction
0,"Also, if everything is interpolated, then the ...",END,[]
1,<https://heapanalytics.com/app/env/1122623983/...,END,[]
2,Maybe later in the week? I believe fixing or g...,END,[]
3,Article with some nice discussion of bad habit...,END,"[""UCZ4VJF6J""]"
4,<https://iceberg.apache.org/docs/latest/aws/#w...,END,[]
...,...,...,...
3930,Let me know if there are any other issues. I m...,END,"[""UU5C7MNMA""]"
3931,Please do! <#CCXPWTVMY|general> makes more sen...,END,[]
3932,"awesome, I'll take a look \n\n\n <https://gitl...",END,[]
3933,"Ah, I do see it has `compiler -&gt; instructio...",END,"[""U016TM9NXEY""]"


In [11]:
nr_valid_with_predictions = pandas.read_json(f'{work_dir}/next_reaction_valid_with_pred.jsonl', lines=True)
nr_valid_with_predictions

,prompt,completion,prediction
0,"Also, if everything is interpolated, then the ...",END,[]
1,<https://heapanalytics.com/app/env/1122623983/...,END,[]
2,Maybe later in the week? I believe fixing or g...,END,[]
3,Article with some nice discussion of bad habit...,END,"[""UCZ4VJF6J""]"
4,<https://iceberg.apache.org/docs/latest/aws/#w...,END,[]
...,...,...,...
3930,Let me know if there are any other issues. I m...,END,"[""UU5C7MNMA""]"
3931,Please do! <#CCXPWTVMY|general> makes more sen...,END,[]
3932,"awesome, I'll take a look \n\n\n <https://gitl...",END,[]
3933,"Ah, I do see it has `compiler -&gt; instructio...",END,"[""U016TM9NXEY""]"


In [35]:
for i in range(len(nr_valid_with_predictions)):
    completion = nr_valid_with_predictions['completion'][i]
    nr_valid_with_predictions.at[i, "test"] = completion.removesuffix(" END").replace("[", "{").replace(']','}')
    prediction = nr_valid_with_predictions['prediction'][i]
    nr_valid_with_predictions.at[i, "pred"] = prediction.replace("[", "{").replace(']','}')
nr_valid_with_predictions

,prompt,completion,prediction,test,pred
0,"Also, if everything is interpolated, then the ...",[] END,"[""UCZ4VJF6J""]",{},"{""UCZ4VJF6J""}"
1,<https://heapanalytics.com/app/env/1122623983/...,[] END,[],{},{}
2,Maybe later in the week? I believe fixing or g...,"[""UCZ4VJF6J""] END","[""U03RNK543HC""]","{""UCZ4VJF6J""}","{""U03RNK543HC""}"
3,Article with some nice discussion of bad habit...,[] END,"[""U012CLUV1KJ""]",{},"{""U012CLUV1KJ""}"
4,<https://iceberg.apache.org/docs/latest/aws/#w...,[] END,"[""U017T5TFW58""]",{},"{""U017T5TFW58""}"
...,...,...,...,...,...
3930,Let me know if there are any other issues. I m...,[] END,[],{},{}
3931,Please do! <#CCXPWTVMY|general> makes more sen...,[] END,[],{},{}
3932,"awesome, I'll take a look \n\n\n <https://gitl...",[] END,[],{},{}
3933,"Ah, I do see it has `compiler -&gt; instructio...",[] END,"[""U016TM9NXEY""]",{},"{""U016TM9NXEY""}"


In [38]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_test_transformed = mlb.fit_transform(nr_valid_with_predictions['test'])
y_pred_transformed = mlb.transform(nr_valid_with_predictions['pred'])

from sklearn.metrics import f1_score
f1 = f1_score(y_test_transformed, y_pred_transformed, average='macro')  # Or 'micro', 'weighted' based on need
f1

## Next Users in Window

For a set of recent messages, try to predict the set of users that might interact next.

In [5]:
nu_completion_separator = " END"

nu_train = pandas.read_json(f'{work_dir}/next_users_in_window_train.jsonl', lines=True)
nu_valid = pandas.read_json(f'{work_dir}/next_users_in_window_valid.jsonl', lines=True)

### With Training Data

In [12]:
examples = 5

random.seed()
pos_count = 0
neg_count = 0

i = 0
while True:
    if pos_count == examples and neg_count == examples:
        break

    i += random.randint(0, 5)
    prompt = nu_train['prompt'][i]
    completion = nu_train['completion'][i]

    pos = completion != f' []{nu_completion_separator}'
    if pos:
        if pos_count < examples:
            pos_count += 1
        else:
            continue
    else:
        if neg_count < examples:
            neg_count += 1
        else: 
            continue

    res = openai.Completion.create(model=next_users_model, prompt=prompt, max_tokens=50, temperature=0, stop=nu_completion_separator)
    choice = res['choices'][0]['text']

    print(f'Choice: {choice}, Actual: {completion}')
    print()

Choice:  ["U012CLUV1KJ"], Actual:  [] END

Choice:  ["U017T5TFW58"], Actual:  ["U016TM9NXEY", "UCZ4VJF6J"] END

Choice:  ["U012CLUV1KJ"], Actual:  ["U012CLUV1KJ"] END

Choice:  ["U012CLUV1KJ"], Actual:  ["U012CLUV1KJ"] END

Choice:  ["U012CLUV1KJ"], Actual:  ["U012CLUV1KJ"] END

Choice:  ["U012CLUV1KJ"], Actual:  [] END

Choice:  ["U012CLUV1KJ"], Actual:  [] END

Choice:  ["UU5C7MNMA"], Actual:  [] END

Choice:  [], Actual:  ["U012CLUV1KJ", "U016TM9NXEY", "UCZ4VJF6J"] END

Choice:  ["U016TM9NXEY"], Actual:  [] END



### With Validation Data

In [6]:
examples = 5

random.seed()
pos_count = 0
neg_count = 0

i = 0
while True:
    if pos_count == examples and neg_count == examples:
        break

    i += random.randint(0, 5)
    prompt = nu_valid['prompt'][i]
    completion = nu_valid['completion'][i]

    pos = completion != f' []{nu_completion_separator}'
    if pos:
        if pos_count < examples:
            pos_count += 1
        else:
            continue
    else:
        if neg_count < examples:
            neg_count += 1
        else: 
            continue

    res = openai.Completion.create(model=next_users_model, prompt=prompt, max_tokens=50, temperature=0, stop=nu_completion_separator)
    choice = res['choices'][0]['text']

    print(f'Choice: {choice}, Actual: {completion}')
    print()

Choice:  [], Actual:  [] END

Choice:  [], Actual:  [] END

Choice:  ["U012CLUV1KJ"], Actual:  [] END

Choice:  ["UCZ4VJF6J"], Actual:  ["U012CLUV1KJ", "UU5C7MNMA"] END

Choice:  ["UCZ4VJF6J"], Actual:  ["U012CLUV1KJ", "UU5C7MNMA"] END

Choice:  ["UCZ4VJF6J"], Actual:  ["U012CLUV1KJ", "UU5C7MNMA"] END

Choice:  [], Actual:  ["U012CLUV1KJ", "U016TM9NXEY", "U017T5TFW58", "U021KG8NMRQ", "UCZ4VJF6J"] END

Choice:  ["U017T5TFW58"], Actual:  ["UCZ4VJF6J", "UU5C7MNMA"] END

Choice:  ["U017T5TFW58"], Actual:  [] END

Choice:  ["U016TM9NXEY"], Actual:  [] END



### Compute Metrics

In [13]:
from IPython.display import clear_output
import multiprocessing

nu_valid_with_predictions = pandas.read_json(f'{work_dir}/next_users_in_window_valid.jsonl', lines=True)

# task that operates on an item
def task(item):
    res = openai.Completion.create(model=next_users_model, prompt=nu_valid_with_predictions['prompt'][item], max_tokens=50, stop=nu_completion_separator)
    nu_valid_with_predictions.at[item, "prediction"] = res['choices'][0]['text']
    return 1

total = len(nu_valid_with_predictions)
items = [i for i in range(total)]
count = 0

# create a process pool that uses all cpus
with multiprocessing.Pool() as pool:
    for result in pool.imap(task, items):
        count +=1
        clear_output(wait=True)
        print(f'Percent Complete: { round(100.0 * count / total) }')

nu_valid_with_predictions.to_json(f'{work_dir}/next_users_in_window_valid_with_pred.jsonl', lines=True, orient='records')
nu_valid_with_predictions

Process SpawnPoolWorker-1038:
Process SpawnPoolWorker-1034:
Process SpawnPoolWorker-1033:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/eric.pinzur/.pyenv/versions/3.9.16/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self

KeyboardInterrupt: 

In [ ]:
nr_valid_with_predictions = pandas.read_json(f'{work_dir}/next_users_in_window_valid_with_pred.jsonl', lines=True)
nr_valid_with_predictions

In [35]:
for i in range(len(nu_valid_with_predictions)):
    completion = nu_valid_with_predictions['completion'][i]
    nu_valid_with_predictions.at[i, "test"] = completion.removesuffix(" END").replace("[", "{").replace(']','}')
    prediction = nu_valid_with_predictions['prediction'][i]
    nu_valid_with_predictions.at[i, "pred"] = prediction.replace("[", "{").replace(']','}')
nu_valid_with_predictions

,prompt,completion,prediction,test,pred
0,"Also, if everything is interpolated, then the ...",[] END,"[""UCZ4VJF6J""]",{},"{""UCZ4VJF6J""}"
1,<https://heapanalytics.com/app/env/1122623983/...,[] END,[],{},{}
2,Maybe later in the week? I believe fixing or g...,"[""UCZ4VJF6J""] END","[""U03RNK543HC""]","{""UCZ4VJF6J""}","{""U03RNK543HC""}"
3,Article with some nice discussion of bad habit...,[] END,"[""U012CLUV1KJ""]",{},"{""U012CLUV1KJ""}"
4,<https://iceberg.apache.org/docs/latest/aws/#w...,[] END,"[""U017T5TFW58""]",{},"{""U017T5TFW58""}"
...,...,...,...,...,...
3930,Let me know if there are any other issues. I m...,[] END,[],{},{}
3931,Please do! <#CCXPWTVMY|general> makes more sen...,[] END,[],{},{}
3932,"awesome, I'll take a look \n\n\n <https://gitl...",[] END,[],{},{}
3933,"Ah, I do see it has `compiler -&gt; instructio...",[] END,"[""U016TM9NXEY""]",{},"{""U016TM9NXEY""}"


In [38]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_test_transformed = mlb.fit_transform(nu_valid_with_predictions['test'])
y_pred_transformed = mlb.transform(nu_valid_with_predictions['pred'])

from sklearn.metrics import f1_score
f1 = f1_score(y_test_transformed, y_pred_transformed, average='macro')  # Or 'micro', 'weighted' based on need
f1

In [30]:
prompt = "For sparrow if I can't use a relative path, I'll need to copy it and check it in -- the protos are recompiled by the build step.\n\nAck. I guess that would be the benefit to have it be a published package?\n\nOne thing that just occurred to me is that it may be a pain to pull that into the Sparrow build. I *think* that everything in the build needs to be in the `sparrow-api` directory (I had to copy the google error details definition into there for things to work).\n\nBut, once this is checked in I can see -- it may be possible to access paths outside of the build"

messages = [
    {
        "role": "system",
        "content": "You trying to predict the user that might be interested in the text from this conversation. The user is speified by a number between 0 and 20 inclusive. If no user is interested, return `nil` instead."
    },
    {
        "role": "user",
        "content": prompt
      }
]

model = "gpt-3.5-turbo"

openai.ChatCompletion.create(model=model, messages=messages, temperature=0, n=1, stop=" end", max_tokens=6 )

<OpenAIObject chat.completion id=chatcmpl-7mPjlIVZO1Mn1OEHQ6760tTRFtBWS at 0x12fa68c70> JSON: {
  "id": "chatcmpl-7mPjlIVZO1Mn1OEHQ6760tTRFtBWS",
  "object": "chat.completion",
  "created": 1691773033,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "User 0"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 189,
    "completion_tokens": 3,
    "total_tokens": 192
  }
}